In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time

from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import Angle, SkyCoord
import astropy.units as u
import geoviews
import pandas as pd
import datashader
import colorcet as cc
from cartopy import crs
from astropy.time import Time
from datetime import datetime

import holoviews as hv
import holoviews.operation.datashader as hd
from holoviews.operation.datashader import rasterize, shade, spread, dynspread, datashade
from holoviews.operation import decimate
import datashader
import pandas as pd
import numpy as np
import datashader as ds
from holoviews import opts

import hvplot.pandas
import bokeh
from bokeh.models import ColumnDataSource, HoverTool
from holoviews.selection import link_selections
from holoviews.plotting.links import DataLink
import param
from tqdm import tqdm

import colorcet as cc

In [ ]:
# read the file

# filename = "/users/elowe/repos/antenna-plots/ant_pos_17AB_plus.parquet"
# filename = "/users/elowe/repos/antenna-plots/modified_ant_pos_17AB_plus_date.parquet"
filename = "/home/scratch/elowe/antenna_data/ant_pos_with_az_al_30k.parquet"
df = pd.read_parquet(filename)
# df = df[::100]
df

In [ ]:
# code to convert the DMJD to date
date = Time(df['DMJD'], format="mjd").to_datetime()
df['date'] = date
df

In [ ]:
 %%time
sc = SkyCoord(df['RAJ2000'], df['DECJ2000'], unit=u.deg, frame="icrs", equinox="J2000")
sc

In [ ]:
%%time

# Get skycoord into a dataframe
galactic_df = sc.galactic.to_table().to_pandas()
galactic_df.tail()

In [ ]:
# Creating one file with all the columns we need



gal_ra = galactic_df['l'].values.tolist()
gal_dec = galactic_df['b'].values.tolist()

df["gal_ra"] = gal_ra
df["gal_dec"] = gal_dec
df1 = df

# antenna_data2.to_parquet('modified_ant_pos_with_az_al_small.parquet', engine='fastparquet')
df1

In [ ]:
df1.to_parquet('modified_ant_pos_17AB_plus.parquet', engine='pyarrow')

In [ ]:
from holoviews import opts
proj = geoviews.Points(df,['RAJ2000', 'DECJ2000'])
proj = proj.opts(opts.Points(tools=['lasso_select','box_select'],projection=crs.Mollweide(),global_extent=True, width=800, height=400))
proj = geoviews.operation.project_points(proj, projection=crs.Mollweide())

# creating the decimated points
# decimated_points = decimate(proj).opts(tools=['lasso_select', 'box_select', 'hover'], projection=crs.Mollweide(), )

hv_shaded = hv.operation.datashader.datashade(rasterize(hv.DynamicMap(proj)), dynamic=True, precompute=True, cmap='fire')
hv_spread = hv.operation.datashader.dynspread(hv_shaded, threshold=.40, max_px=1)
hv_spread = hv_spread.opts(tools=['lasso_select','help'],bgcolor='white',projection=crs.Mollweide(), global_extent=True, frame_width=800, frame_height=400)

In [ ]:
hv_spread